In [41]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
import time
import pdfplumber
import io
import re

In [42]:
def get_internal_links(url):
    html = requests.get(url, timeout=10).text
    soup = BeautifulSoup(html, "html.parser")

    links = set()
    base_domain = urlparse(url).netloc

    for a in soup.find_all("a", href=True):
        full_url = urljoin(url, a["href"])
        if urlparse(full_url).netloc == base_domain:
            links.add(full_url)

    return links


In [49]:
def extract_html_text(html):
    soup = BeautifulSoup(html, "html.parser")

    for tag in soup(["script", "style", "noscript"]):
        tag.decompose()

    if not soup.body:
        return None

    return soup.body.get_text(" ", strip=True)

def clean_pdf_text(text: str) -> str:
    if not text:
        return ""

    # 1. Normalize line endings
    text = text.replace("\r\n", "\n").replace("\r", "\n")

    # 2. Remove control characters (keep \n for now)
    text = re.sub(r'[\x00-\x08\x0B-\x1F\x7F]', '', text)

    # 3. Remove spaces around line breaks
    text = re.sub(r'[ \t]*\n[ \t]*', '\n', text)

    # Remove long dot leaders (TOC / layout artifacts)
    text = re.sub(r'\.{3,}', ' ', text)

    # 4. Collapse multiple newlines (paragraph breaks)
    text = re.sub(r'\n{3,}', '\n\n', text)

    # 5. Merge lines inside paragraphs
    text = re.sub(r'(?<!\n)\n(?!\n)', ' ', text)

    # 6. Normalize whitespace
    text = re.sub(r'[ \t]+', ' ', text)

    # 7. Fix space before punctuation
    text = re.sub(r'\s+([.,;:!?])', r'\1', text)

    return text.strip()


def extract_pdf_text(response):
    pages_text = []

    with pdfplumber.open(io.BytesIO(response.content)) as pdf:
        for page in pdf.pages:
            page_text = page.extract_text()
            if page_text:
                pages_text.append(page_text)

    raw_text = "\n\n".join(pages_text)
    return clean_pdf_text(raw_text)



def extract_text(url):
    r = requests.get(url, timeout=15)

    content_type = r.headers.get("Content-Type", "").lower()

    if "text/html" in content_type:
        return extract_html_text(r.text)

    if "application/pdf" in content_type:
        return extract_pdf_text(r)

    return None


In [52]:
import time

start_url = "https://portugal2030.pt/"

visited = set()
to_visit = {start_url}
all_text = {}

page_counter = 0          # counter for pages
start_time = time.time()  # start timer

while to_visit:
    url = to_visit.pop()
    if url in visited:
        continue

    try:
        page_counter += 1
        print(f"[{page_counter}] Scraping: {url}")

        text = extract_text(url)
        if not text:
            continue

        all_text[url] = text

        new_links = get_internal_links(url)
        to_visit |= new_links

        visited.add(url)
        time.sleep(1)  # be polite

    except Exception as e:
        print("Failed:", url, e)

# After crawling, show elapsed time
end_time = time.time()
elapsed = end_time - start_time
print(f"\nScraped {page_counter} pages in {elapsed:.2f} seconds.")


[1] Scraping: https://portugal2030.pt/
[2] Scraping: https://portugal2030.pt/noticias/
[3] Scraping: https://portugal2030.pt/politica-de-privacidade/
[4] Scraping: https://portugal2030.pt/contactos
[5] Scraping: https://portugal2030.pt/politica-de-acessibilidade/
[6] Scraping: https://portugal2030.pt/2025/12/16/portal-mais-transparencia-com-nova-informacao-sobre-fundos-europeus/
[7] Scraping: https://portugal2030.pt/en/politica-de-acessibilidade/
[8] Scraping: https://portugal2030.pt/en/calls/
[9] Scraping: https://portugal2030.pt/2025/12/03/ja-comecou-a-mostra-dos-fundos-europeus/
[10] Scraping: https://portugal2030.pt/documentos/
[11] Scraping: https://portugal2030.pt/termos-e-condicoes/
[12] Scraping: https://portugal2030.pt/2025/12/09/excelencia-inovacao-e-impacto-conheca-todos-os-vencedores-dos-premios-dos-fundos-europeus/
[13] Scraping: https://portugal2030.pt/2025/12/19/projeto-inovador-melhora-eficiencia-e-conforto-no-tratamento-de-fraturas-osseas/
[14] Scraping: https://portug

Cannot set gray non-stroke color because /'P328' is an invalid float value
Cannot set gray non-stroke color because /'P341' is an invalid float value


[22] Scraping: https://portugal2030.pt/programas/
[23] Scraping: https://portugal2030.pt/en/portugal-2030/o-que-e-o-portugal-2030/#enquadramento
[24] Scraping: https://portugal2030.pt/#main-content
[25] Scraping: https://portugal2030.pt/eventos-arquivo/webinar-sobre-projetos-de-transicao-verde-para-entidades-portuguesas/
[26] Scraping: https://portugal2030.pt/wp-content/uploads/sites/3/2025/12/Reprogramacao_Programas_AcaoClimaticaSustentabilidade.pdf
[27] Scraping: https://portugal2030.pt/resultados-pt2030/
[28] Scraping: https://portugal2030.pt/plano-anual-de-avisos/
[29] Scraping: https://portugal2030.pt/aviso-2024/sistema-de-incentivos-de-base-territorial-it-cim-terras-de-tras-os-montes/
[30] Scraping: https://portugal2030.pt/wp-content/uploads/sites/3/2025/10/ALGARVE2030_agost25.pdf
[31] Scraping: https://portugal2030.pt/aviso-2024/descarbonizacao-do-porto-de-portimao/
[32] Scraping: https://portugal2030.pt/eventos-arquivo/sessao-sobre-aviso-know4euf-estudos-para-reforcar-o-conheci

Cannot set gray non-stroke color because /'Pattern1' is an invalid float value
Cannot set gray non-stroke color because /'Pattern2' is an invalid float value
Cannot set gray non-stroke color because /'Pattern3' is an invalid float value


[190] Scraping: https://portugal2030.pt/wp-content/uploads/sites/3/2025/09/unnamed-file.pdf-157.octet-stream
[191] Scraping: https://portugal2030.pt/aviso-2024/estagios-profissionais-2o-aviso/
[192] Scraping: https://portugal2030.pt/en/portugal-2030/o-que-e-o-portugal-2030/#prioridades
[193] Scraping: https://portugal2030.pt/aviso-2024/apoios-a-contratacao-jovens-ii/
[194] Scraping: https://portugal2030.pt/aviso-2024/acoes-inovadoras-para-responder-ao-desafio-demografico-iti-agua-e-ecossistemas-de-paisagem/
[195] Scraping: https://portugal2030.pt/o-portugal-2030/programas/#outros
[196] Scraping: https://portugal2030.pt/programas/#outros
[197] Scraping: https://portugal2030.pt/wp-content/uploads/sites/3/2024/07/Newsletter-Portugal-2030-numero-8.pdf
[198] Scraping: https://portugal2030.pt/aviso-2024/bolsas-de-ensino-superior-para-alunos-carenciados-ano-letivo-2025-2026/
[199] Scraping: https://portugal2030.pt/aviso-2024/incubadoras-de-empresas-de-base-nao-tecnologica/
[200] Scraping: htt

...

Cannot set gray non-stroke color because /'P92' is an invalid float value
Cannot set gray non-stroke color because /'P94' is an invalid float value
Cannot set gray non-stroke color because /'P96' is an invalid float value
Cannot set gray non-stroke color because /'P98' is an invalid float value
Cannot set gray non-stroke color because /'P99' is an invalid float value
Cannot set gray non-stroke color because /'P100' is an invalid float value
Cannot set gray non-stroke color because /'P102' is an invalid float value
Cannot set gray non-stroke color because /'P104' is an invalid float value
Cannot set gray non-stroke color because /'P106' is an invalid float value
Cannot set gray non-stroke color because /'P111' is an invalid float value
Cannot set gray non-stroke color because /'P113' is an invalid float value
Cannot set gray non-stroke color because /'P115' is an invalid float value
Cannot set gray non-stroke color because /'P116' is an invalid float value
Cannot set gray non-stroke col

[1953] Scraping: https://portugal2030.pt/2023/11/10/ate-14-de-novembro-vote-no-projeto-aldeias-de-xisto-finalista-dos-premios-regiostars-2023/
[1954] Scraping: https://portugal2030.pt/legislacao/decreto-lei-n-o-140-2013-de-18-de-outubro-versao-consolidada/
[1955] Scraping: https://portugal2030.pt/wp-content/uploads/sites/3/2025/03/DLB-homologacao-4_2025Ass.pdf
[1956] Scraping: https://portugal2030.pt/aviso-2024/acolhimento-de-requerentes-e-beneficiarios-de-protecao-internacional/#main-content
[1957] Scraping: https://portugal2030.pt/legislacao/decreto-regulamentar-regional-n-o-25-2025-a/#main-content
[1958] Scraping: https://portugal2030.pt/wp-content/uploads/sites/3/2023/06/retificacaoreg2021_1060.pdf
[1959] Scraping: https://portugal2030.pt/ajuda-arquivo/quais-sao-os-programas-do-pt2030/#main-content
[1960] Scraping: https://portugal2030.pt/2024/03/25/programa-euro-med-recruta-diretor-de-informacao-e-comunicacao/#main-content
[1961] Scraping: https://portugal2030.pt/wp-content/upload

Cannot set gray non-stroke color because /'Pattern1' is an invalid float value
Cannot set gray non-stroke color because /'Pattern2' is an invalid float value
Cannot set gray non-stroke color because /'Pattern3' is an invalid float value
Cannot set gray non-stroke color because /'Pattern1' is an invalid float value
Cannot set gray non-stroke color because /'Pattern2' is an invalid float value
Cannot set gray non-stroke color because /'Pattern3' is an invalid float value


[2073] Scraping: https://portugal2030.pt/wp-content/uploads/sites/3/2025/01/Deb_CIC_2a-Reprogramacao_FAMI.pdf
[2074] Scraping: https://portugal2030.pt/2023/03/02/assista-as-sessoes-sobre-a-1-a-convocatoria-do-programa-sudoe-e-a-2-a-do-europe/
[2075] Scraping: https://portugal2030.pt/legislacao/projeto-de-deliberacao-n-o-4-2025-pl/#main-content
[2076] Scraping: https://portugal2030.pt/portugal-2030/?doing_wp_cron=1660905030.0215570926666259765625#programas
[2077] Scraping: https://portugal2030.pt/legislacao/decreto-lei-n-o-37-a-2024/
[2078] Scraping: https://portugal2030.pt/legislacao/deliberacao-n-o-23-2024-pl/
[2079] Scraping: https://portugal2030.pt/wp-content/uploads/sites/3/2025/09/unnamed-file.pdf-231.octet-stream
[2080] Scraping: https://portugal2030.pt/legislacao/decisao-de-execucao-ue-no-2021-1129/
[2081] Scraping: https://portugal2030.pt/legislacao/decreto-lei-n-o-32-2024-2/#main-content
[2082] Scraping: https://portugal2030.pt/wp-content/uploads/sites/3/2025/09/unnamed-file.p

KeyboardInterrupt: 

In [51]:
all_text

{'https://portugal2030.pt/wp-content/uploads/sites/3/2022/05/20220304_APPortugal2030_vs-detalhada_SUBM.pdf': 'Versão 2.0 do Acordo de Parceria 4 de março de 2022 1\n\n© MINISTÉRIO DO PLANEAMENTO AP PT2030 1.0 (informal): 15.nov.2021 Consulta pública: 15.nov a 30.nov.21 Observações COM consulta inter-serviços: 19.jan.2022 AP PT 2030 2.0: aprovação em RCM de 03.mar.2022 AP PT 2030 2.0 (formal): 04.mar.2022 Rua Professor Gomes Teixeira, 2, 8.º 1399–022 LISBOA • Telefone: (+351) 213 927 372 http://www.portugal.gov.pt 2\n\nÍNDICE Introdução 7 1. Portugal 2030 – os fundos para a transformação do País no quadro da Estratégia Portugal 2030 9 1.1. Síntese da evolução da transformação estrutural impulsionada pelos fundos europeus 9 1.2. Alinhamento do Acordo de Parceria com a Estratégia Portugal 2030 e complementaridade com outros instrumentos 13 1.3. Estrutura Programática do Portugal 2030 20 2. Principais Opções Programáticas por Objetivo Estratégico 25 2.1. Objetivo Estratégico 1 – Portugal m